# 🧭 11강: AI 역량 진단 & 성장

> 30문항 자기진단 → 레이더 차트 시각화 → 90일 성장 로드맵 자동 생성

In [ ]:
# ⚙️ 환경 설정
!pip install -q -U google-genai

from google import genai
from google.colab import userdata
import getpass
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'DejaVu Sans'

try:
    api_key = userdata.get('GEMINI_API_KEY')
    if isinstance(api_key, dict):
        api_key = str(list(api_key.values())[0])
    API_KEY = str(api_key).strip()
except Exception:
    API_KEY = getpass.getpass('🔑 Gemini API Key를 입력하세요: ')

client = genai.Client(api_key=API_KEY)
MODEL = 'gemini-2.5-flash'
print('✅ 준비 완료!')

## 🔬 실습 1: 30문항 자기 진단 생성

In [ ]:
prompt1 = """
너는 AI 교육 전문가이자 역량 진단 설계사야.

학원 원장의 AI 역량을 진단하는 설문지를 만들어줘.
6개 영역, 각 5문항, 총 30문항.

영역: 프롬프트 엔지니어링 / AI 도구 활용 / 데이터 분석 /
자동화 설계 / 콘텐츠 제작 / 전략적 사고

각 문항:
Q. [구체적 상황 질문]
1) 전혀 못한다 (1점)
2) 도움 있으면 가능 (2점)
3) 혼자 할 수 있다 (3점)
4) 남을 가르칠 수 있다 (4점)

학원 업무 연결 구체적 문항으로.
"""

response = client.models.generate_content(model=MODEL, contents=prompt1)
survey = response.text
print('📋 AI 역량 자기 진단 (30문항)')
print('='*50)
print(survey)

## 🔬 실습 2: 레이더 차트 생성

> 아래 점수를 수정한 후 실행하세요!

In [ ]:
# ✏️ 여러분의 진단 점수를 입력하세요! (최소 5, 최대 20)
my_scores = {
    'Prompt': 14,
    'Tools': 16,
    'Data': 8,
    'Automation': 6,
    'Content': 18,
    'Strategy': 10
}

total = sum(my_scores.values())
if total <= 48:
    level = 'Lv.1 🌱 인식자'
elif total <= 72:
    level = 'Lv.2 🌿 활용자'
elif total <= 96:
    level = 'Lv.3 🌳 설계자'
else:
    level = 'Lv.4 🏔️ 전략가'

print(f'📊 총점: {total}/120')
print(f'🎯 레벨: {level}')
print(f'💪 가장 강한 영역: {max(my_scores, key=my_scores.get)}')
print(f'📈 개선 필요 영역: {min(my_scores, key=my_scores.get)}')

In [ ]:
# 📊 레이더 차트
categories = list(my_scores.keys())
values = list(my_scores.values())
max_val = 20

angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]
values_closed = values + [values[0]]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

ax.plot(angles, [max_val] * len(angles), 'b--', alpha=0.2, label='Max (20)')
ax.fill(angles, [max_val] * len(angles), alpha=0.05, color='blue')
ax.plot(angles, values_closed, 'o-', linewidth=2, color='#FF6B6B', label='My Score')
ax.fill(angles, values_closed, alpha=0.25, color='#FF6B6B')

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, size=12, fontweight='bold')
ax.set_ylim(0, max_val)
ax.set_title(f'AI Competency Radar\nTotal: {total}/120 | {level}',
             size=16, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))

plt.tight_layout()
plt.savefig('ai_radar.png', dpi=150, bbox_inches='tight')
plt.show()
print('📊 레이더 차트 저장 완료!')

## 🔬 실습 3: 90일 성장 로드맵

In [ ]:
weakest = min(my_scores, key=my_scores.get)
strongest = max(my_scores, key=my_scores.get)

prompt3 = f"""
너는 AI 학습 설계 코치야.

아래 역량 진단 결과를 바탕으로 90일 AI 성장 로드맵을 작성해줘.

진단 결과:
- 전체 레벨: {level}
- 총점: {total}/120
- 가장 약한 영역: {weakest} ({my_scores[weakest]}점)
- 가장 강한 영역: {strongest} ({my_scores[strongest]}점)
- 전체 점수: {my_scores}

| 기간 | 목표 | 학습 내용 | 실습 과제 | 확인 방법 |
|---|---|---|---|---|
| 1~30일 | 약한 영역 보강 | ... | ... | ... |
| 31~60일 | 중간 영역 강화 | ... | ... | ... |
| 61~90일 | 통합 실전 | ... | ... | ... |

하루 30분 이내. 약한 영역 먼저 보강.
90일 후 목표 점수 설정.
"""

response = client.models.generate_content(model=MODEL, contents=prompt3)
roadmap = response.text
print('🗺️ 90일 성장 로드맵')
print('='*50)
print(roadmap)

In [ ]:
# 💾 저장
output = f"진단결과: {level} ({total}/120)\n{my_scores}\n\n로드맵:\n{roadmap}"
with open('AI역량_진단결과.txt', 'w', encoding='utf-8') as f:
    f.write(output)
from google.colab import files
files.download('AI역량_진단결과.txt')
files.download('ai_radar.png')
print('✅ 다운로드!')

## 🏆 11강 완료!
👉 [12강: 통합 대시보드](https://colab.research.google.com/github/Reasonofmoon/cedu-1/blob/main/notebooks/12_cockpit.ipynb)